## Projet AGIR : modèle de prédiction
#### Rappel de l'objectif
On cherche à prédire si l'évolution en proportion du DPE ou du climat peut influer la consommation électrique, pour permettre à l'Etat d'évaluer la mise en place de politiques sur le DPE ou en anticipation de l'évolution climatique.

#### Prérequis
Afin que le code fonctionne au mieux, faites en sorte de respecter les 3 conditions de mon fichier requirements.txt. Vous pouvez utiliser d'autres versions mais à votre péril ;)

#### Outil proposé
Dans un premier temps, bien relire ce qui a été marqué sur le drive, puis lire le module model.py, qui propose les 3 fonctions : create_model_data, train_model, use_model

#### Etape 1 : jeu de données

Il faudra évidemment le réadapter pour avoir les colonnes souhaitées à l'entraînement. Ici j'ouvre très simplement un fichier à titre d'exemple.

In [ ]:
import pandas as pd
from model import ModelCreator

In [ ]:
data = pd.read_csv("data-final-02_04.csv")
data.columns

### Préface au modèle : objet ModelCreator

Cet objet est la solution la plus simple pour conserver les données d'une fonction à l'autre. Lisez bien la documentation pour la comprendre.

#### Etape 2 : normalisation des données

La fonction create_model_data va se charger de normaliser les données. Le seul travail à faire de votre côté est la distinction entre les variable quantitatives (scalaires, donc numériques et unidimensionnelles), et les variables catégorielle (autrement dit qualitatives)

In [ ]:
mc = ModelCreator(data)
mc.create_model_data(column_to_predict="Consommation annuelle moyenne par logement de l'adresse (MWh)", quantitative_variables=["Centile_25%", "Centile_75%"], categorical_variables=["N°_région_(BAN)"])

#### Etape 3 : Entraînement d'un modèle MLPRegressor

Un MLPRegressor est un réseau de neurones. Un réseau de neurones est une sucession de fonctions polynomiales enveloppées chacune d'une fonction dite "d'activation", la résultante étant une grande fonction. Chaque facteur constant du polynôme est inconnu et est l'objet de l'entraînement. L'entrée de la fonction sont les variables quantitatives et catégorielles choisies, et la sortie de la fonction est la "column_to_predict". Dans notre cas, il s'agit d'un modèle de régression, dans le sens où l'entraînement porte sur une variable quantitative (à l'opposé d'un classifieur où la sortie est une variable catégorielle).

Je propose dans le module de contrôler la "taille" de cette fonction. Chaque élément de la liste "network_layer" ajoute un polynôme, et la valeur indique la taille du polynôme. La valeur max_iterations limite le temps d'entraînement si le modèle à du mal à converger, c'est à dire parvenir à un stade de l'entraînement où le modèle n'évolue plus de façon significative.

Les résultats de la fonction se découpent en 3 parties :
    - La loss curve. C'est une série d'un calcul que l'on appelle la perte pour un modèle. Cette perte mesure le taux d'erreur à l'entraînement, et guide également l'entraînement, car l'ajustement du modèle est fait pour minimiser cette perte. Elle peut être tracée, sachant qu'une courbe de perte idéale est quand elle décroît régulièrement jusqu'à atteindre un plancher (converge) le plus proche de 0 possible
    - Le score d'entraînement. C'est une métrique arbitraire pour mesurer l'efficacité concréte du modèle, basé sur la formule du R². Cette formule pénalise fortement les écarts marqués à la réalité attendue.
    - Le score de test. Pareil que la précédente, mais cette fois appliqué au jeu de données qui a été mis à l'écart de l'entraînement. Il permet de contrôler que le modèle n'a pas juste "appris par coeur" les données d'entraînement.

Nommons le score d'entraînement St celui de test Se :
    - St sera généralement supérieur à Se. On cherche au mieux à faire en sorte que St = Se
    - Se = St = 1 indique que le modèle est parfait. C'est l'utopie qui est visée par nos modèles.
    - Se = 0 indique que le modèle ne fait pas mieux que si on prenait simplement la moyenne des valeurs connues de sortie pour prédire.
    - Se < 0 indique que le modèle est moins performant que la moyenne. Il est alors inenvisageable d'utiliser le modèle.

Je ne sais pas ce que permettra le jeu de données, mais avec le peu de temps qu'on a dépasser Se > 0.50 sera déjà une très bonne preuve de potentiel. Mais ne vous focalisez pas dessus. Se > 0 est la vraie base à viser.

In [ ]:
loss_curve, score_train, score_test = mc.train_model(max_iterations=30)
print(min(loss_curve), score_train, score_test)

#### Etape 4 : prédiction

Si vous avez un modèle qui marche, vous pourrez proposer une prédiction. L'entrée doit être présentée sous forme d'un dictionnaire. Si vous voulez faire plusieurs prédictions en utilisant toutes les fonctionnalités de ma fonction, demandez moi ou observez le code que j'ai fait.

In [ ]:
prediction_input = {
    "Centile_25%": 1,
    "Centile_75%": 10,
    "N°_région_(BAN)": 11
}
mc.use_model(prediction_input)